<a href="https://colab.research.google.com/github/Aakaey181/RA_Porject_ML1/blob/main/app_withdrag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install streamlit streamlit-drawable-canvas
!pip install streamlit pyngrok

In [15]:
!pip install streamlit streamlit-cropper pyngrok

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!pkill -f streamlit

In [36]:
%%writefile app1.py
import streamlit as st
from PIL import Image
from streamlit_cropper import st_cropper
import numpy as np
import tensorflow as tf

# Load your model (adjust path as needed)
model = tf.keras.models.load_model("drive/MyDrive/ra/trained_models/clean_mobilenet_binary_classifier.h5")

st.title("Zebra Crossing Detector (Cropper Demo)")

uploaded_file = st.file_uploader("Upload an image", type=["jpg","jpeg","png", "webp"])
if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.write("Original uploaded image:")
    st.image(image, use_container_width=True)

    st.write("Use the crop box below to select the region of interest:")
    # real-time update => you see changes as you drag
    # box_color => color of bounding box
    # aspect_ratio=None => free aspect ratio
    cropped_img = st_cropper(
        image,
        realtime_update=False,
        box_color="red",
        aspect_ratio=None
    )

    st.write("Cropped image preview:")
    st.image(cropped_img, use_container_width=True)

    if st.button("Predict on Cropped Region"):
        # Preprocess
        resized = cropped_img.resize((224,224))
        arr = np.array(resized).astype("float32") / 255.0
        arr = np.expand_dims(arr, axis=0)

        prob = model.predict(arr)[0][0]
        conf = prob * 100
        st.write(f"Confidence: {conf:.2f}% that a zebra crossing is present.")


Overwriting app1.py


In [37]:
!ngrok authtoken 2vCjd1FQD3nj46KDGwA02cD8fmE_2JU7HFoujPjiYnTW9xDDR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [38]:
!nohup streamlit run app1.py &
from pyngrok import ngrok

# Specify the protocol as http for the tunnel
public_url = ngrok.connect(8501, "http")

print("Public URL:", public_url)

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://124b-34-106-85-94.ngrok-free.app" -> "http://localhost:8501"
